In [1]:
from elasticsearch import Elasticsearch
import urllib3

urllib3.disable_warnings    

es = Elasticsearch(hosts=['https://localhost:9200'],basic_auth=('elastic','bvxL*B3U4Ks8yk2bETD6'),verify_certs=False, ssl_show_warn=False)

es.ping()

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


True

In [2]:
es.indices.create(index='another_one')

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'another_one'})

In [5]:
es.indices.exists(index='another_one')

HeadApiResponse(True)

In [20]:
doc = {"first_name":"hai","last_name":"bai"}

es.index(index='another_one',id=3,document=doc)

ObjectApiResponse({'_index': 'another_one', '_id': '3', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1})

In [7]:
# es.delete(index='another_one',id=3)

ObjectApiResponse({'_index': 'another_one', '_id': '3', '_version': 2, 'result': 'deleted', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1})

In [21]:
body = {
    "match_all":{}
}
es.search(index='another_one',query=body)

ObjectApiResponse({'took': 564, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'another_one', '_id': '3', '_score': 1.0, '_source': {'first_name': 'hai', 'last_name': 'bai'}}]}})

In [22]:
#Quering in 8.2.2

q = {
    "match":{
        "last_name":"bai"
    }
}
res =es.search(index='another_one',query=q)
res['hits']['hits'][0]['_source']


{'first_name': 'hai', 'last_name': 'bai'}

In [24]:
#Update By query

q = {
    
    "match": {
        "first_name":"hai"
    }}

s = {
        "lang": "painless",
        "source": "ctx._source.last_name='haibai'",
    }
                



try:
    res = es.update_by_query(index='another_one',conflicts="proceed",refresh=True,query=q,script=s)
    print(res)
except Exception as e:
    print(e)

{'took': 251, 'timed_out': False, 'total': 1, 'updated': 1, 'deleted': 0, 'batches': 1, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []}


In [25]:
body = {
    "match_all":{}
}
es.search(index='another_one',query=body)

ObjectApiResponse({'took': 2, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'another_one', '_id': '3', '_score': 1.0, '_source': {'last_name': 'haibai', 'first_name': 'hai'}}]}})

In [28]:
q = {
    "last_name":"haibai"
}
es.update(index='another_one',id=3,doc=q)

ObjectApiResponse({'_index': 'another_one', '_id': '3', '_version': 2, 'result': 'noop', '_shards': {'total': 0, 'successful': 0, 'failed': 0}, '_seq_no': 3, '_primary_term': 1})

In [12]:
q = {
    "term":{
        "last_name":"haibai"
    }
}


es.delete_by_query(index='another_one',query=q)

ObjectApiResponse({'took': 19, 'timed_out': False, 'total': 1, 'deleted': 1, 'batches': 1, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []})

In [31]:
q = {"match_all":{

}}

res = es.search(index='another_one',query=q)
res['hits']['hits']

[{'_index': 'another_one',
  '_id': '3',
  '_score': 1.0,
  '_source': {'last_name': 'haibai', 'first_name': 'hai'}}]

In [33]:
doc_mapping = {
    'properties':{
    'name':{'type':'text'},
    'age' :{'type':'integer'},
    'dumb':{'type':'boolean'}
}}

es.indices.create(index='indexformapping',mappings=doc_mapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'indexformapping'})

In [34]:
es.indices.exists(index='indexformapping')

HeadApiResponse(True)

In [35]:
docu = {
    'name':'Shantanu Kumar',
    'age':  23,
    'dumb':True

}
es.index(index='indexformapping',document=docu,id=4)

ObjectApiResponse({'_index': 'indexformapping', '_id': '4', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

In [36]:
es.indices.delete(index='indexformapping')

ObjectApiResponse({'acknowledged': True})

In [37]:
es.indices.get_alias(index='*')

/tmp/ipykernel_8447/1934033638.py:1: ElasticsearchWarning: this request accesses system indices: [.security-7], but in a future major version, direct access to system indices will be prevented by default
  es.indices.get_alias(index='*')


ObjectApiResponse({'.security-7': {'aliases': {'.security': {'is_hidden': True}}}, 'cities': {'aliases': {}}, 'english': {'aliases': {}}, 'another_one': {'aliases': {}}})

In [39]:
ma = {
    
}
es.indices.put_mapping(index='another_one',source=ma)

ObjectApiResponse({'acknowledged': True})